In [1]:
import os
import json

This notebook creates a dataset (images and labels as a json file). The dataset created can be used for pose classification.   
In order to create a new dataset for gesture recoginition specify the following parameters 

**no_of_classes** - Number of classes to be created. i.e. For hand pose the number of hand gestures to be created.

**path_dir** - Path to the directory to be created

**dataset_name** - The name of the dataset to be created


In [2]:
def create_directories_for_classes(no_of_classes, path_dir, dataset_name):
    dir_ = os.path.join(path_dir, dataset_name)
    for i in range(no_of_classes):
        dir_to_create = os.path.join(dir_,"%s" % (i+1))
        try:
            os.makedirs(dir_to_create)
        except FileExistsError:
            print(os.path.join("The following directory was not created because it already exsists", dir_ , ))


In [29]:
dir_datasets = '/home/mikyas/mike_dataset/jj/'
dataset_name = "hand_dataset"
no_of_classes = 5
create_directories_for_classes(no_of_classes, dir_datasets, dataset_name )

In [30]:
import ipywidgets.widgets as widgets
dir_ = os.path.join(dir_datasets, dataset_name)
curr_class_no = 1
button_layout = widgets.Layout(width='128px', height='32px')
curr_dir = os.path.join(dir_,'%s'%curr_class_no )
collecting_button = widgets.Button(description= 'Collect Class ' + str(curr_class_no), button_style='success', layout=button_layout)
prev_button = widgets.Button(description='Previous Class', button_style='primary', layout=button_layout)
nxt_button = widgets.Button(description='Next Class', button_style='info', layout=button_layout)

dir_count = widgets.IntText(layout=button_layout, value=len(os.listdir(curr_dir)))
dir_count.continuous_update

False

In [31]:
from uuid import uuid1
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_w.value)
def save_dir():
    global curr_dir, dir_count
    save_snapshot(curr_dir)
    dir_count.value = len(os.listdir(curr_dir))
def prev_dir():
    global curr_class_no, curr_dir, no_of_classes
    if curr_class_no>1:
        curr_class_no-=1
    curr_dir = os.path.join(dir_,'%s'%curr_class_no )
    collecting_button.description = 'Collect Class ' + str(curr_class_no)
    dir_count.value = len(os.listdir(curr_dir))
    dir_count.continuous_update
def nxt_dir():
    global curr_class_no, curr_dir, no_of_classes
    if curr_class_no<no_of_classes:
        curr_class_no+=1
    curr_dir = os.path.join(dir_,'%s'%curr_class_no )
    collecting_button.description = 'Collect Class ' + str(curr_class_no)
    dir_count.value = len(os.listdir(curr_dir))

        

collecting_button.on_click(lambda x: save_dir())
nxt_button.on_click(lambda x: nxt_dir())
prev_button.on_click(lambda x: prev_dir())


In [7]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
WIDTH = 256
HEIGHT = 256
camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30, capture_device=1)
#camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=50)

camera.running = True

In [8]:
import ipywidgets
from IPython.display import display

image_w = ipywidgets.Image(format='jpeg', width=256, height=256)


In [32]:
display(image_w)
display(widgets.HBox([dir_count, collecting_button]))
display(widgets.HBox([ nxt_button]))
display(widgets.HBox([ prev_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [10]:
def execute(change):
    image = change['new']
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])    

In [22]:
execute({'new': camera.value})

In [23]:
camera.observe(execute, names='value')

In [13]:
camera.unobserve_all()
camera.running = False

In [33]:
def generate_labels(dir_, dataset_name):
    labels = []
    starting_label = 1
    for i in range(len(os.listdir(dir_))):
        dir_to_check = os.path.join(dir_,"%s" % (i+1))
        for j in range(len(os.listdir(dir_to_check))):
            labels.append(starting_label)
        starting_label+=1
    labels_to_dict = {"labels": labels}
    with open((dir_+'.json'), 'w') as outfile:
        json.dump(labels_to_dict, outfile)
    return labels      

In [34]:
def rename_images(dir_):
    overall_count = 0
    #dir_ = dir_+dataset_name
    for i in range(len(os.listdir(dir_))):
        dir_to_check = os.path.join(dir_,"%s" % (i+1))
        dir_to_check+='/'
        for count, filename in enumerate(os.listdir(dir_to_check)):
            dst = "%08d.jpg"% overall_count
            src = dir_to_check+filename
            dst = dir_to_check+dst 
            os.rename(src, dst)
            overall_count+=1

In [35]:
generate_labels(dir_, dataset_name)
rename_images(dir_)

In [36]:
import shutil
dir_training = dir_datasets +'/trainig/'
try:
    os.makedirs(dir_training)
except FileExistsError:
    print(os.path.join("The following directory was not created because it already exsists", dir_ , ))
for i in range(len(os.listdir(dir_))):
    dir_to_check = os.path.join(dir_,"%s" % (i+1))+'/'
    for count, filename in enumerate(os.listdir(dir_to_check)):
            src = dir_to_check+filename
            shutil.move(src,dir_training)
    os.rmdir(dir_to_check)
shutil.move(dir_training,dir_)
shutil.move(dir_+'.json',dir_)

5
/home/mikyas/mike_dataset/jj/hand_dataset
0
1
2
3
4


'/home/mikyas/mike_dataset/jj/hand_dataset/hand_dataset.json'

In [21]:
camera.running = True